<a href="https://colab.research.google.com/github/ysg1202/automoblile/blob/main/0725_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [2]:
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # [B, 3, 224, 224] → [B, 16, 224, 224]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # → [B, 16, 112, 112]

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                             # → [B, 32, 56, 56]

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))                 # → [B, 64, 1, 1]
        )
        self.classifier = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.classifier(x)
        return x


In [8]:
# ✅ 설정
data_dir = '/content/data'
batch_size = 16
num_epochs = 100
lr = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ✅ 데이터셋
train_dataset = datasets.ImageFolder(f"{data_dir}/train", transform=transform)
val_dataset = datasets.ImageFolder(f"{data_dir}/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

num_classes = len(train_dataset.classes)

# ✅ 모델 정의
model = CustomCNN(num_classes=num_classes).to(device)

# ✅ 손실 함수 & 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# ✅ 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"[Epoch {epoch+1}/{num_epochs}] Loss: {total_loss:.4f}")

# ✅ 저장
torch.save(model.state_dict(), "custom_cnn.pth")
print("✅ 커스텀 모델 저장 완료")


FileNotFoundError: Couldn't find any class folder in /content/data/train.